In [2]:
#Cell 1 - Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.inspection import partial_dependence
from xgboost import XGBRegressor
import shap
from scipy.stats import randint, uniform
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualization
plt.style.use('default')
sns.set_theme(style="whitegrid")

In [3]:
# Cell 2 - Data Loading Function
def load_and_clean_data(file_path: str) -> pd.DataFrame:
    """
    Load and clean the air quality dataset with improved handling of missing values
    and feature engineering.
    """
    header = [
        "No", "year", "month", "day", "hour", "pm2.5", "DEWP",
        "TEMP", "PRES", "cbwd", "Iws", "Is", "Ir"
    ]
    df = pd.read_csv(file_path, names=header, delimiter=',', skiprows=1)
    print(f"Initial data shape: {df.shape}")
    
    # Convert numeric columns
    df = df[pd.to_numeric(df['year'], errors='coerce').notnull()]
    numeric_cols = ['year', 'month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print("\nMissing values before cleaning:")
    print(df.isnull().sum())
    
    # Create datetime and additional temporal features
    df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df['season'] = df['month'].apply(lambda x: (x%12 + 3)//3)
    df['is_weekend'] = df['date'].dt.dayofweek.isin([5, 6]).astype(int)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    
    # Handle missing values
    df = df.dropna(subset=['pm2.5']).sort_values('date').reset_index(drop=True)
    
    print("\nMissing values after cleaning:")
    print(df.isnull().sum())
    print(f"Final data shape: {df.shape}")
    
    return df

In [4]:
# Cell 3 - Feature Engineering Function:

def create_advanced_features(df):
    """Create advanced features for the model"""
    print("Creating advanced features...")
    
    # Time-based features
    df['hour_of_day'] = df['hour']
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month_of_year'] = df['month']
    
    # Cyclical encoding
    for col, max_val in [('hour_of_day', 24), ('day_of_week', 7), ('month_of_year', 12)]:
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col]/max_val)
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    
    # Weather interactions
    df['humid_temp'] = df['TEMP'] * df['DEWP']
    df['wind_pressure'] = df['PRES'] * df['Iws']
    
    # Rolling statistics
    windows = [6, 12, 24, 48]
    for col in ['TEMP', 'DEWP', 'PRES', 'Iws']:
        for window in windows:
            df[f'{col}_rolling_mean_{window}h'] = df[col].rolling(window=window, min_periods=1).mean()
            df[f'{col}_rolling_std_{window}h'] = df[col].rolling(window=window, min_periods=1).std()
    
    # Lag features
    lag_hours = [24, 48, 72]
    for lag in lag_hours:
        df[f'pm25_lag_{lag}h'] = df['pm2.5'].shift(lag)
    
    print(f"Number of features after engineering: {len(df.columns)}")
    return df

In [5]:
# Cell 4 - Preprocessor Function

def create_preprocessor():
    """
    Create a preprocessing pipeline for both numerical and categorical features.
    """
    # Define numeric features based on actual columns after feature engineering
    numeric_features = [
        'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir', 
        'season', 'is_weekend', 'hour_sin', 'hour_cos',
        'hour_of_day', 'day_of_week', 'month_of_year',
        'hour_of_day_sin', 'hour_of_day_cos',
        'day_of_week_sin', 'day_of_week_cos',
        'month_of_year_sin', 'month_of_year_cos',
        'humid_temp', 'wind_pressure'
    ]
    
    # Add rolling statistics features
    for col in ['TEMP', 'DEWP', 'PRES', 'Iws']:
        for window in [6, 12, 24, 48]:
            numeric_features.append(f'{col}_rolling_mean_{window}h')
            numeric_features.append(f'{col}_rolling_std_{window}h')
    
    # Add lag features
    for lag in [24, 48, 72]:
        numeric_features.append(f'pm25_lag_{lag}h')
    
    categorical_features = ['cbwd']
    
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(drop='first', sparse_output=False))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop'
    )
    
    return preprocessor

In [6]:
# Cell 5 - Test Initial Setup

# Load and process data
file_path = 'PRSA_data_2010.1.1-2014.12.31.csv'
df = load_and_clean_data(file_path)

# Create advanced features
df = create_advanced_features(df)

# Check the features we created
print("\nColumns in processed dataframe:")
print(df.columns.tolist())

# Create preprocessor and check its structure
preprocessor = create_preprocessor()
print("\nPreprocessor structure:")
# Print the transformers and their associated columns
for name, transformer, columns in preprocessor.transformers:
    print(f"\nTransformer: {name}")
    print("Columns:")
    print(columns)

# Quick check for any missing values after feature engineering
print("\nMissing values after feature engineering:")
print(df.isnull().sum().sum())

# Print shape of data
print(f"\nFinal dataframe shape: {df.shape}")

Initial data shape: (43825, 13)

Missing values before cleaning:
No          0
year        0
month       0
day         0
hour        0
pm2.5    2067
DEWP        0
TEMP        0
PRES        0
cbwd        0
Iws         0
Is          0
Ir          0
dtype: int64

Missing values after cleaning:
No            0
year          0
month         0
day           0
hour          0
pm2.5         0
DEWP          0
TEMP          0
PRES          0
cbwd          0
Iws           0
Is            0
Ir            0
date          0
season        0
is_weekend    0
hour_sin      0
hour_cos      0
dtype: int64
Final data shape: (41757, 18)
Creating advanced features...
Number of features after engineering: 64

Columns in processed dataframe:
['No', 'year', 'month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir', 'date', 'season', 'is_weekend', 'hour_sin', 'hour_cos', 'hour_of_day', 'day_of_week', 'month_of_year', 'hour_of_day_sin', 'hour_of_day_cos', 'day_of_week_sin', 'day_of_week_c

In [7]:
# Cell 6 - Data Preperation 

# Drop rows with NaN values after feature engineering
df = df.dropna().reset_index(drop=True)

# Prepare features and target
feature_cols = [col for col in df.columns if col not in ['pm2.5', 'date', 'No', 'year', 'month', 'day', 'hour']]
X = df[feature_cols]
y = df['pm2.5']

# Print feature information
print(f"Number of features used: {len(feature_cols)}")
print(f"Number of samples: {len(X)}")

# Check if all required features are present
preprocessor = create_preprocessor()
all_features = []
for _, _, columns in preprocessor.transformers:
    all_features.extend(columns)

missing_features = set(all_features) - set(X.columns)
if missing_features:
    print("\nWarning: Missing features:")
    print(missing_features)
else:
    print("\nAll required features are present in the dataset.")

# Initialize cross-validation
tscv = TimeSeriesSplit(n_splits=5, gap=24)


Number of features used: 57
Number of samples: 41685

All required features are present in the dataset.


In [20]:
# Cell 7 - Time Series Model Evaluation Pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.base import clone
import shap
from sklearn.model_selection import ParameterGrid
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor


# Initialize models with parameter grids based on professor's requirements
models = {
    'Lasso': (
        Lasso(random_state=42),
        {
            'regressor__alpha': [0.001, 0.01, 0.1, 1.0],
            'regressor__max_iter': [10000],
        }
    ),
    'Ridge': (
        Ridge(random_state=42),
        {
            'regressor__alpha': [0.001, 0.01, 0.1, 1.0],
        }
    ),
    'RandomForest': (
        RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=100),
        {
            'regressor__max_depth': [1, 3, 10, 30, 100, 300],
            'regressor__min_samples_split': [2, 3, 10, 30],
            'regressor__max_features': [3, 5, 7, None]
        }
    ),
    'XGBoost': (
        XGBRegressor(
            random_state=42,
            n_jobs=-1,
            learning_rate=0.03,
            n_estimators=10000,
            early_stopping_rounds=50,
            enable_categorical=True,
            colsample_bytree=0.9,
            subsample=0.66,
            missing=np.nan,
            eval_metric='rmse'
        ),
        {
            'regressor__max_depth': [1, 3, 10, 30],
            'regressor__reg_alpha': [0.01, 1, 100],
            'regressor__reg_lambda': [0.01, 1, 100]
        }
    )
}

def get_train_val_split(X_other, y_other):
    """Split other data into train and validation maintaining temporal order"""
    n = len(X_other)
    train_size = int(0.8 * n)  # Use 80% for training, 20% for validation
    
    X_train = X_other.iloc[:train_size]
    X_val = X_other.iloc[train_size:]
    y_train = y_other.iloc[:train_size]
    y_val = y_other.iloc[train_size:]
    
    return X_train, X_val, y_train, y_val

def evaluate_model(X, y, model_name, model, param_grid, n_splits=4):
    """Evaluate a single model using time series cross-validation"""
    if model_name == 'XGBoost':
        X = X.copy()
        categorical_columns = X.select_dtypes(include=['object']).columns
        for col in categorical_columns:
            X[col] = X[col].astype('category')
            
        print("\nXGBoost Configuration:")
        print("Base parameters:", model.get_params())
        print("\nParameter grid being searched:", param_grid)
    
    # Stage 1: TimeSeriesSplit with 4 folds
    tscv = TimeSeriesSplit(n_splits=4)
    test_scores = []
    best_params_list = []
    all_predictions = []
    all_true_values = []
    
    print(f"\nEvaluating {model_name}...")
    
    for fold, (other_idx, test_idx) in enumerate(tscv.split(X), 1):
        # Stage 1: Split into other and test sets
        X_other, X_test = X.iloc[other_idx], X.iloc[test_idx]
        y_other, y_test = y.iloc[other_idx], y.iloc[test_idx]
        
        # Stage 2: Split other into train and validation
        X_train, X_val, y_train, y_val = get_train_val_split(X_other, y_other)
        
        best_score = float('inf')
        best_params = None
        best_model = None
        
        # Grid search without cross-validation
        if model_name == 'XGBoost':
            preprocessor = create_preprocessor()
            preprocessor.fit(pd.concat([X_train, X_val, X_test]))
            
            X_train_processed = preprocessor.transform(X_train)
            X_val_processed = preprocessor.transform(X_val)
            X_test_processed = preprocessor.transform(X_test)
            
            print(f"\nFold {fold} - XGBoost Training:")
            print(f"Training set shape: {X_train_processed.shape}")
            print(f"Validation set shape: {X_val_processed.shape}")
            
            for params in ParameterGrid(param_grid):
                xgb_model = clone(model)
                xgb_model.set_params(**{k.replace('regressor__', ''): v for k, v in params.items()})
                
                xgb_model.fit(
                    X_train_processed, y_train,
                    eval_set=[(X_val_processed, y_val)],
                    verbose=False
                )
                
                val_pred = xgb_model.predict(X_val_processed)
                val_score = np.sqrt(mean_squared_error(y_val, val_pred))
                
                if val_score < best_score:
                    best_score = val_score
                    best_params = params
                    best_model = Pipeline([
                        ('preprocessor', preprocessor),
                        ('regressor', xgb_model)
                    ])
            
            print(f"Best parameters for fold {fold}:", best_params)
            print(f"Best validation RMSE: {best_score:.2f}")
            
        else:
            for params in ParameterGrid(param_grid):
                pipeline = Pipeline([
                    ('preprocessor', create_preprocessor()),
                    ('regressor', clone(model))
                ])
                pipeline.set_params(**params)
                
                pipeline.fit(X_train, y_train)
                val_pred = pipeline.predict(X_val)
                val_score = np.sqrt(mean_squared_error(y_val, val_pred))
                
                if val_score < best_score:
                    best_score = val_score
                    best_params = params
                    best_model = pipeline
        
        # Make predictions on test set and store score
        test_pred = best_model.predict(X_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
        
        test_scores.append(test_rmse)
        best_params_list.append(best_params)
        all_predictions.extend(test_pred)
        all_true_values.extend(y_test)
        
        print(f"Fold {fold} - Test RMSE: {test_rmse:.2f}")
    
    # Calculate mean and std of test scores
    mean_score = np.mean(test_scores)
    std_score = np.std(test_scores)
    print(f"Overall Performance - RMSE: {mean_score:.2f} ± {std_score:.2f}")
    
    if model_name == 'XGBoost':
        print("\nSummary of best parameters across folds:")
        for fold, params in enumerate(best_params_list, 1):
            print(f"Fold {fold}: {params}")
    
    return {
        'test_scores': test_scores,
        'best_params': best_params_list,
        'predictions': np.array(all_predictions),
        'true_values': np.array(all_true_values),
        'final_model': best_model,
        'mean_score': mean_score,
        'std_score': std_score
    }

# Calculate baseline scores
def calculate_baseline(y, n_splits=4):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    baseline_scores = []
    
    for train_idx, test_idx in tscv.split(y):
        y_train = y.iloc[train_idx]
        y_test = y.iloc[test_idx]
        # Use mean of training set as prediction
        baseline_pred = [y_train.mean()] * len(y_test)
        baseline_scores.append(np.sqrt(mean_squared_error(y_test, baseline_pred)))
    
    return np.mean(baseline_scores), np.std(baseline_scores)

# Run evaluation
baseline_mean, baseline_std = calculate_baseline(y)
print(f"Baseline RMSE: {baseline_mean:.2f} ± {baseline_std:.2f}")

results = {}
for model_name, (model, param_grid) in models.items():
    results[model_name] = evaluate_model(
        X, y, model_name, model, param_grid)

Baseline RMSE: 92.09 ± 4.36

Evaluating Lasso...
Fold 1 - Test RMSE: 72.10
Fold 2 - Test RMSE: 66.59
Fold 3 - Test RMSE: 60.80
Fold 4 - Test RMSE: 65.87
Overall Performance - RMSE: 66.34 ± 4.00

Evaluating Ridge...
Fold 1 - Test RMSE: 72.08
Fold 2 - Test RMSE: 66.91
Fold 3 - Test RMSE: 60.78
Fold 4 - Test RMSE: 64.81
Overall Performance - RMSE: 66.14 ± 4.07

Evaluating RandomForest...
Fold 1 - Test RMSE: 69.88
Fold 2 - Test RMSE: 67.35
Fold 3 - Test RMSE: 60.15
Fold 4 - Test RMSE: 61.18
Overall Performance - RMSE: 64.64 ± 4.09

XGBoost Configuration:
Base parameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.9, 'device': None, 'early_stopping_rounds': 50, 'enable_categorical': True, 'eval_metric': 'rmse', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.03, 'max_bin': None, 'max_

In [21]:
def create_model_summary_table():
    summary_data = {
        'Model': [],
        'Hyperparameter': [],
        'Values': []
    }
    
    # Lasso
    summary_data['Model'].append('Lasso')
    summary_data['Hyperparameter'].append('alpha')
    summary_data['Values'].append('[0.001, 0.01, 0.1, 1.0]')
    summary_data['Model'].append('Lasso')
    summary_data['Hyperparameter'].append('max_iter')
    summary_data['Values'].append('[10000]')
    
    # Ridge
    summary_data['Model'].append('Ridge')
    summary_data['Hyperparameter'].append('alpha')
    summary_data['Values'].append('[0.001, 0.01, 0.1, 1.0]')
    
    # Random Forest
    rf_params = ['n_estimators', 'max_depth', 'min_samples_split', 'max_features']
    rf_values = ['100', '[1, 3, 10, 30, 100, 300]', '[2, 3, 10, 30]', '[3, 5, 7, None]']
    for param, value in zip(rf_params, rf_values):
        summary_data['Model'].append('Random Forest')
        summary_data['Hyperparameter'].append(param)
        summary_data['Values'].append(value)
    
    # XGBoost
    xgb_base_params = ['learning_rate', 'n_estimators', 'early_stopping_rounds', 'colsample_bytree', 'subsample']
    xgb_base_values = ['0.03', '10000', '50', '0.9', '0.66']
    for param, value in zip(xgb_base_params, xgb_base_values):
        summary_data['Model'].append('XGBoost')
        summary_data['Hyperparameter'].append(param)
        summary_data['Values'].append(value)
        
    xgb_tuned_params = ['max_depth', 'reg_alpha', 'reg_lambda']
    xgb_tuned_values = ['[1, 3, 10, 30]', '[0.01, 1, 100]', '[0.01, 1, 100]']
    for param, value in zip(xgb_tuned_params, xgb_tuned_values):
        summary_data['Model'].append('XGBoost')
        summary_data['Hyperparameter'].append(param)
        summary_data['Values'].append(value)
    
    # Create DataFrame
    summary_df = pd.DataFrame(summary_data)
    
    # Display table with styling
    return (summary_df.style
            .set_properties(**{'text-align': 'left'})
            .set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'left'),
                                           ('background-color', '#f2f2f2'),
                                           ('padding', '8px')]},
                {'selector': 'td', 'props': [('padding', '8px')]}
            ])
            .hide(axis='index'))

# Display the summary table
model_summary = create_model_summary_table()
display(model_summary)

# Save the table as a high-resolution figure
plt.figure(figsize=(10, 8), dpi=300)
ax = plt.subplot(111, frame_on=False)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

# Create table
table = plt.table(cellText=pd.DataFrame(model_summary.data).values,
                 colLabels=['Model', 'Hyperparameter', 'Values'],
                 cellLoc='left',
                 loc='center',
                 colWidths=[0.2, 0.3, 0.5])

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.5)

# Save figure
plt.savefig('model_hyperparameters_summary.png', 
            dpi=300,
            bbox_inches='tight',
            format='png')
plt.close()

Model,Hyperparameter,Values
Lasso,alpha,"[0.001, 0.01, 0.1, 1.0]"
Lasso,max_iter,[10000]
Ridge,alpha,"[0.001, 0.01, 0.1, 1.0]"
Random Forest,n_estimators,100
Random Forest,max_depth,"[1, 3, 10, 30, 100, 300]"
Random Forest,min_samples_split,"[2, 3, 10, 30]"
Random Forest,max_features,"[3, 5, 7, None]"
XGBoost,learning_rate,0.03
XGBoost,n_estimators,10000
XGBoost,early_stopping_rounds,50


In [33]:
# Cell 8 - Enhanced Results Analysis and Model Interpretation
import shap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

def create_improved_force_plots(explainer, shap_values, X_processed, feature_names, indices=[0, 100, 200]):
    """Create improved SHAP force plots with enhanced readability and captions"""
    plt.style.use('default')
    
    for idx in indices:
        plt.figure(figsize=(15, 5))
        
        shap.force_plot(
            explainer.expected_value,
            shap_values[idx],
            X_processed[idx],
            feature_names=feature_names,
            matplotlib=True,
            show=False,
            text_rotation=45,
            contribution_threshold=0.05
        )
        
        plt.title(f'SHAP Force Plot: Feature Contributions for Sample {idx}', pad=20, fontsize=14)
        plt.tight_layout()
        
        # Add detailed caption
        caption = (f'Feature contribution analysis for sample {idx}. Positive values (red) '
                  'increase the prediction, while negative values (blue) decrease it.')
        plt.figtext(0.5, 0.02, caption, ha='center', fontsize=10, wrap=True)
        
        plt.savefig(f'force_plot_{idx}.png', dpi=300, bbox_inches='tight', pad_inches=0.5)
        plt.close()
def analyze_results(results, baseline_mean, baseline_std):
    """
    Enhanced analysis function with improved visualizations and SHAP analysis
    """
    # 1. Performance Comparison Plot
    plt.figure(figsize=(12, 6))
    models = list(results.keys())
    means = [results[m]['mean_score'] for m in models]
    stds = [results[m]['std_score'] for m in models]
    
    plt.errorbar(range(len(models)), means, yerr=stds, fmt='o', capsize=5,
                markersize=8, elinewidth=2, capthick=2)
    plt.axhline(y=baseline_mean, color='r', linestyle='--', 
                label=f'Baseline ({baseline_mean:.2f} ± {baseline_std:.2f})')
    plt.xticks(range(len(models)), models, rotation=45)
    plt.xlabel('Model Type')  # Added x-axis label
    plt.ylabel('RMSE (μg/m³)')
    plt.title('Model Performance Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    plt.savefig('model_performance.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Best Model Analysis (XGBoost was best)
    best_model_results = results['XGBoost']
    
    plt.figure(figsize=(12, 8))
    plt.scatter(best_model_results['true_values'], 
               best_model_results['predictions'], 
               alpha=0.5, color='blue', s=20, 
               label='Predictions')
    
    max_val = max(max(best_model_results['true_values']), 
                  max(best_model_results['predictions']))
    min_val = min(min(best_model_results['true_values']), 
                  min(best_model_results['predictions']))
    
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', 
             lw=2, label='Perfect Prediction')
    
    z = np.polyfit(best_model_results['true_values'], 
                   best_model_results['predictions'], 1)
    p = np.poly1d(z)
    plt.plot(best_model_results['true_values'], 
             p(best_model_results['true_values']), 
             "g-", alpha=0.8, 
             label=f'Trend Line (y = {z[0]:.2f}x + {z[1]:.2f})')
    
    r2 = r2_score(best_model_results['true_values'], 
                  best_model_results['predictions'])
    plt.text(0.05, 0.95, f'R² = {r2:.3f}', 
             transform=plt.gca().transAxes, 
             fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
    
    plt.xlabel('Actual PM2.5 (μg/m³)')
    plt.ylabel('Predicted PM2.5 (μg/m³)')
    plt.title('XGBoost - Actual vs Predicted PM2.5 Values')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('actual_vs_predicted.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Error Distribution
    errors = best_model_results['predictions'] - best_model_results['true_values']
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=50, alpha=0.75, color='blue')
    plt.axvline(x=0, color='r', linestyle='--', label='Zero Error')
    plt.title('Distribution of XGBoost Prediction Errors')
    plt.xlabel('Prediction Error (μg/m³)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('error_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Error Analysis by PM2.5 Level
    plt.figure(figsize=(10, 6))
    plt.scatter(best_model_results['true_values'], 
               errors, 
               alpha=0.5, s=20)
    plt.axhline(y=0, color='r', linestyle='--', label='Zero Error')
    plt.xlabel('Actual PM2.5 (μg/m³)')
    plt.ylabel('Prediction Error (μg/m³)')
    plt.title('Error Analysis by PM2.5 Level')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig('error_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 5. SHAP Analysis
    if hasattr(best_model_results['final_model']['regressor'], 'feature_importances_'):
        preprocessor = best_model_results['final_model']['preprocessor']
        feature_names = (preprocessor.named_transformers_['num'].get_feature_names_out().tolist() +
                        preprocessor.named_transformers_['cat'].get_feature_names_out().tolist())
        
        explainer = shap.TreeExplainer(best_model_results['final_model']['regressor'])
        X_processed = best_model_results['final_model']['preprocessor'].transform(X.iloc[:1000])
        shap_values = explainer.shap_values(X_processed)
        
        plt.figure(figsize=(12, 8))
        shap.summary_plot(
            shap_values, 
            X_processed,
            feature_names=feature_names,
            max_display=10,
            plot_size=(12, 8),
            show=False
        )
        plt.title('Top 10 Features by SHAP Value Impact', pad=20)
        plt.tight_layout()
        plt.savefig('shap_summary.png', dpi=300, bbox_inches='tight')
        plt.close()

        create_improved_force_plots(explainer, shap_values, X_processed, feature_names)

    # Print model comparison results
    comparison_df = pd.DataFrame({
        'Model': models,
        'RMSE': [f"{results[m]['mean_score']:.2f} ± {results[m]['std_score']:.2f}" for m in models],
        'Improvement': [f"{((baseline_mean - results[m]['mean_score']) / baseline_mean * 100):.1f}%" for m in models],
        'Best Parameters': [
            ', '.join(f"{k.split('__')[-1]}={v}" for k, v in results[m]['best_params'][-1].items())
            for m in models
        ]
    })

    print("\nModel Performance Summary")
    print("=" * 100)
    print(comparison_df.to_string(index=False))
    
    return comparison_df

# Generate results
summary_df = analyze_results(results, baseline_mean, baseline_std)


Model Performance Summary
       Model         RMSE Improvement                                    Best Parameters
       Lasso 66.34 ± 4.00       28.0%                          alpha=1.0, max_iter=10000
       Ridge 66.14 ± 4.07       28.2%                                          alpha=1.0
RandomForest 64.64 ± 4.09       29.8% max_depth=100, max_features=7, min_samples_split=3
     XGBoost 61.13 ± 4.27       33.6%           max_depth=3, reg_alpha=1, reg_lambda=100


In [29]:
def create_analysis_summary_tables():
    """Create and format analysis summary tables with proper styling"""
    # Table 1: Model Interpretability Steps
    steps_df = pd.DataFrame({
        'Step': ['1', '2', '3', '4', '5'],
        'Analysis Type': [
            'Global Feature Importance',
            'Local Feature Impact',
            'Error Analysis',
            'Model Performance Visualization',
            'Cross-validation'
        ],
        'Description': [
            'SHAP analysis with feature importance visualization',
            'Individual prediction explanations with force plots',
            'Error distribution and PM2.5 level dependency analysis',
            'Actual vs Predicted with residual analysis',
            'Temporal stability assessment with cross-validation'
        ]
    })
    
    # Table 2: Key Findings with actual results and proper formatting
    findings_df = pd.DataFrame({
        'Category': [
            'Model Performance',
            'Model Performance',
            'Model Performance',
            'Model Parameters',
            'Model Parameters',
            'Model Behavior',
            'Model Behavior'
        ],
        'Finding': [
            f"XGBoost achieved best performance (RMSE: {61.13:.2f} ± {4.27:.2f})",
            f"Significant improvement over baseline ({92.09:.2f} ± {4.36:.2f})",
            "All models exceeded baseline by >28%",
            "XGBoost optimal with max_depth=3",
            "High L2 regularization (reg_lambda=100) preferred",
            "Enhanced accuracy in later time periods",
            "Consistent cross-validation performance"
        ]
    })
    
    # Apply consistent styling with proper captions
    for df, name, caption in [(steps_df, 'steps', 'Model Interpretation Steps and Methodology'),
                             (findings_df, 'findings', 'Key Model Performance Findings')]:
        plt.figure(figsize=(12, 6), dpi=300)
        ax = plt.subplot(111, frame_on=False)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
        table = plt.table(
            cellText=df.values,
            colLabels=df.columns,
            cellLoc='left',
            loc='center',
            colWidths=[0.1, 0.3, 0.6] if name == 'steps' else [0.3, 0.7]
        )
        
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.5)
        
        plt.title(caption, pad=20)
        plt.savefig(f'model_analysis_{name}.png',
                   dpi=300,
                   bbox_inches='tight',
                   format='png')
        plt.close()

In [32]:
def plot_xgboost_importance(results):
    """Create XGBoost feature importance visualizations with visible captions"""
    best_model = results['XGBoost']['final_model']
    xgb_model = best_model['regressor']
    booster = xgb_model.get_booster()
    
    # Get feature names with proper formatting
    preprocessor = best_model['preprocessor']
    actual_feature_names = np.concatenate([
        preprocessor.named_transformers_['num'].get_feature_names_out(),
        preprocessor.named_transformers_['cat'].get_feature_names_out()
    ])
    
    name_map = {f'f{i}': name for i, name in enumerate(actual_feature_names)}
    
    def plot_importance(importance_dict, metric_name):
        # Convert and sort importance scores
        renamed_importance = {name_map[k]: v for k, v in importance_dict.items()}
        sorted_items = sorted(renamed_importance.items(), key=lambda x: x[1], reverse=True)
        top_features = [x[0] for x in sorted_items[:10]]
        top_scores = [x[1] for x in sorted_items[:10]]
        
        # Create figure with extra space for caption
        plt.figure(figsize=(12, 7))  # Increased height for caption
        
        # Create main axis for the plot, leaving space at bottom for caption
        ax = plt.gca()
        ax.set_position([0.1, 0.15, 0.85, 0.75])  # [left, bottom, width, height]
        
        # Create bar plot
        bars = ax.barh(range(len(top_features)), top_scores, color='skyblue')
        
        # Format feature names for readability
        ax.set_yticks(range(len(top_features)))
        ax.set_yticklabels([name.replace('_', ' ').title() for name in top_features])
        
        # Add value labels
        for i, bar in enumerate(bars):
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2,
                   f'{width:.2f}',
                   ha='left', va='center', fontsize=10)
        
        # Add labels and title
        ax.set_xlabel(f'Importance Score ({metric_name})')
        ax.set_title(f'Top 10 Most Important Features in PM2.5 Prediction ({metric_name})')
        ax.grid(axis='x', linestyle='--', alpha=0.7)
        
        # Add caption at the bottom
        caption_text = {
            'weight': 'Number of times a feature appears in the trees.',
            'gain': 'Average gain of splits which use this feature.',
            'cover': 'Average coverage of splits which use this feature.',
            'total_gain': 'Total gain of splits which use this feature.',
            'total_cover': 'Total coverage of splits which use this feature.'
        }
        
        plt.figtext(0.1, 0.02, 
                   f"Caption: {caption_text[metric_name]} Higher scores indicate stronger influence on predictions.",
                   wrap=True, 
                   ha='left', 
                   va='bottom',
                   fontsize=10)
        
        # Save figure
        plt.savefig(f'xgboost_importance_{metric_name}.png',
                   dpi=300,
                   bbox_inches='tight',
                   pad_inches=0.2)  # Added padding to prevent caption cutoff
        plt.close()

    # Generate plots for all importance metrics
    for metric in ['weight', 'gain', 'cover', 'total_gain', 'total_cover']:
        importance = booster.get_score(importance_type=metric)
        plot_importance(importance, metric)